In [49]:
import pandas as pd

In [50]:
df = pd.read_excel('data/База данных.xlsx', sheet_name=None)
print(df.keys())

dict_keys(['Продажи', 'СПР_стоимость_доставки', 'СПР_даты_заказа', 'СПР_продукты', 'СПР_покупатели', 'СПР_Менеджеры'])


In [51]:
sales = df.get('Продажи')
shipping_cost = df.get('СПР_стоимость_доставки') #
date_order = df.get('СПР_даты_заказа') #
products = df.get('СПР_продукты') #
supporters = df.get('СПР_покупатели') #
menagers = df.get('СПР_Менеджеры')


In [52]:
sales_values = {}
a = sales
for col in a.columns:
    tv = a[col].nunique()
    sales_values[col] = tv
sales_values['total'] = a.shape[0]
sales_values = pd.Series(sales_values)
sales_values


Order_ID             571
Приоритет              5
Количество            50
Выручка              728
Метод доставки         2
Прибыль              721
Покупатель           173
Регион                 6
Продукт              445
Упаковка продукта      3
total                730
dtype: int64

In [53]:
sales.columns

Index(['Order_ID', 'Приоритет', 'Количество', 'Выручка', 'Метод доставки',
       'Прибыль', 'Покупатель', 'Регион', 'Продукт', 'Упаковка продукта'],
      dtype='object')

In [54]:
menagers

,Регион,Менеджер
0,Nunavut,Петров Иван
1,Northwest Territories,Иванов Петр
2,Atlantic,Семейкин Владислав
3,West,Мерингольд Ванесса
4,Ontario,Крестьянин Арчибальд
5,Prarie,Конюхов Добромир


In [55]:
sales_01 = pd.merge(sales, shipping_cost, how='left', on='Метод доставки')
sales_02 = pd.merge(sales_01, date_order, how='left', on='Order_ID')
sales_03 = pd.merge(sales_02, products, how='left', on='Продукт')
sales_04 = pd.merge(sales_03, supporters, how='left', on='Покупатель')
sales_full = pd.merge(sales_04, menagers, how='left', on='Регион')

In [56]:
sales_full.head()

,Order_ID,Приоритет,Количество,Выручка,Метод доставки,Прибыль,Покупатель,Регион,Продукт,Упаковка продукта,Стоимость доставки,Дата,Категория продукта,Подкатегория продукта,Сегмент покупателя,Менеджер
0,643,High,21,2 781.82,Express Air,-695.26,Monica Federle,Nunavut,"SAFCO Commercial Wire Shelving, Black",Large Box,10,2022-01-02,Office Supplies,Storage & Organization,Corporate,Петров Иван
1,8995,High,35,3 389.93,Express Air,737.94,Beth Paige,Northwest Territories,Luxo Professional Combination Clamp-On Lamps,Large Box,10,2022-01-15,Furniture,Office Furnishings,Consumer,Иванов Петр
2,9127,Not Specified,7,2 039.56,Express Air,-329.49,Bryan Davis,Northwest Territories,"Tennsco Snap-Together Open Shelving Units, Sta...",Large Box,10,2022-01-16,Office Supplies,Storage & Organization,Corporate,Иванов Петр
3,26272,Low,6,905.94,Express Air,-4.19,Carlos Daly,Northwest Territories,Hon Olson Stacker Stools,Large Box,10,2022-02-24,Furniture,Chairs & Chairmats,Home Office,Иванов Петр
4,26272,Low,6,905.94,Express Air,-4.19,Carlos Daly,Northwest Territories,Hon Olson Stacker Stools,Large Box,10,2022-02-24,Furniture,Chairs & Chairmats,Corporate,Иванов Петр


In [57]:
sales_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Order_ID               875 non-null    int64         
 1   Приоритет              875 non-null    object        
 2   Количество             875 non-null    int64         
 3   Выручка                875 non-null    object        
 4   Метод доставки         875 non-null    object        
 5   Прибыль                875 non-null    object        
 6   Покупатель             875 non-null    object        
 7   Регион                 875 non-null    object        
 8   Продукт                875 non-null    object        
 9   Упаковка продукта      875 non-null    object        
 10  Стоимость доставки     875 non-null    int64         
 11  Дата                   875 non-null    datetime64[ns]
 12  Категория продукта     875 non-null    object        
 13  Подка

In [62]:
sales_full['Выручка'] = sales_full['Выручка'].str.replace('\xa0', '').astype(float)
sales_full['Прибыль'] = sales_full['Прибыль'].str.replace('\xa0', '').astype(float)
sales_full['Дата'] = pd.to_datetime(sales_full['Дата'])
# sales_full['Дата']

In [65]:
sales_full['month'] = sales_full['Дата'].dt.month

In [91]:
test = sales_full.groupby(['month','Менеджер']).agg(
    viruchka = ('Выручка', 'sum'),
    pribil = ('Прибыль', 'sum'),
    zakazov = ('Order_ID', 'count'),
    logistic = ('Стоимость доставки', 'sum'),
    mean_cheak = ('Выручка', 'mean'),
    akb = ('Покупатель', 'nunique')
).reset_index()
test['mean_cheak'] = round(test['mean_cheak'], 2)
test.loc['total'] = [
    "",
    "",
    test['viruchka'].sum(),
    test['pribil'].sum(),
    test['zakazov'].sum(),
    test['logistic'].sum(),
    test['mean_cheak'].mean(),
    test['akb'].sum()
    ]
test

,month,Менеджер,viruchka,pribil,zakazov,logistic,mean_cheak,akb
0,1,Иванов Петр,51980.00,10836.79,23,145,2260.000,16
1,1,Конюхов Добромир,8801.21,146.11,15,90,586.750,8
2,1,Крестьянин Арчибальд,12306.98,2766.39,12,60,1025.580,6
3,1,Мерингольд Ванесса,155949.91,22027.73,83,500,1878.910,28
4,1,Петров Иван,37130.27,45480.75,35,215,1060.860,20
5,1,Семейкин Владислав,73160.02,10840.62,49,280,1493.060,22
6,2,Иванов Петр,73017.17,18784.03,45,280,1622.600,24
7,2,Конюхов Добромир,22656.01,3312.98,16,120,1416.000,7
8,2,Крестьянин Арчибальд,3687.20,-267.62,8,50,460.900,4
9,2,Мерингольд Ванесса,63562.16,8809.22,65,380,977.880,24


In [86]:
sales_full['Прибыль'].nun

0      -695.26
1       737.94
2      -329.49
3        -4.19
4        -4.19
        ...   
870    1886.41
871       5.00
872     310.22
873     -14.35
874      30.29
Name: Прибыль, Length: 875, dtype: float64